# Making a version of *Quantum Rogues*

[*Quantum Rogues*](https://github.com/qiskit-community/qiskit-camp-europe-19/issues/5) was a game created at the [Qiskit Camp Europe](https://qiskit.camp/europe/) hackathon in September 2019. In this notebook we create a new game, which takes the basic idea of the original *Quantum Rogues* and ports it to the PewPew.

In [ ]:
%matplotlib notebook

In this game, the player controls a flying avatar that can move around a room. I think of it as a bat - a qubat, in fact - but you can interpret it as you wish.

First we simply create the room and the bat, and set up the controls. The **O** button makes the bat flap its wings. The ◀︎ and ► buttons move left and right.

In [ ]:
import pew

pew.init()
screen = pew.Pix()

# create a border of B=2 pixels
for A in range(8):
    for B in [0,7]:
        screen.pixel(A,B,2)
        screen.pixel(B,A,2) 

# the player is a B=3 pixel
X,Y = 4,6
screen.pixel(X,Y,3)

while True: # loop which checks for user input and responds

    # use key presses to determine how the player moves
    dX,dY = 0,1 # default is to fall
    keys = pew.keys()
    if keys!=0:
        if keys&pew.K_O: # fly up with O
            dY = -1
        # just left and right
        if keys&pew.K_LEFT:
            dX = -1
            dY = 0
        if keys&pew.K_RIGHT:
            dX = +1
            dY = 0
    
    # blank player pixel at old pos
    screen.pixel(X,Y,0)
    # change pos
    if Y+dY in range(1,7):
        Y+=dY
    if X+dX in range(1,7):
        X+=dX
    # put player pixel at new pos
    screen.pixel(X,Y,3)
    
    pew.show(screen)

    pew.tick(1/6)

Each room will have doors, through which it can move to neighbouring rooms. There are four possible doors: one in each direction.

Whether a door is open or closed is generated randomly from a two qubit quantum circuit. The left door depends on the result from one of the qubits when a z measurement is made: `0` for open and `1` for closed. The right door depends similarly on the z measurement outcome of the other qubit. The doors on the floor and ceiling depend on the x measurement outcomes.

The program below adds in these doors, and ends whenever you enter one.

In [ ]:
import pew
from microqiskit import *

pew.init()
screen = pew.Pix()

# set positions of doors
l = (0,4)
r = (7,4)
u = (3,0)
d = (3,7)
doors = [l,r,u,d]

# create a border of B=2 pixels
for A in range(8):
    for B in [0,7]:
        screen.pixel(A,B,2)
        screen.pixel(B,A,2) 

# the player is a B=3 pixel
X,Y = 4,6
screen.pixel(X,Y,3)

# set up the circuit that decides whether doors are open
qc = QuantumCircuit(2,2)

# and those for measurement
meas_zz = QuantumCircuit(2,2)
meas_zz.measure(0,0)
meas_zz.measure(1,1)
meas_xx = QuantumCircuit(2,2)
meas_xx.h(0)
meas_xx.h(1)
meas_xx.measure(0,0)
meas_xx.measure(1,1)

# use the results to set which doors are open
m_zz = simulate(qc+meas_zz,shots=1,get='memory')[0]
m_xx = simulate(qc+meas_xx,shots=1,get='memory')[0]

opened = []
if m_zz[0]=='0':
    opened.append(l)
if m_zz[1]=='0':
    opened.append(r)
if m_xx[0]=='0':
    opened.append(u)
if m_xx[1]=='0':
    opened.append(d)
    
# set open door pixels to B=0
for door in doors:
    screen.pixel(door[0],door[1],2)
for door in opened:
    screen.pixel(door[0],door[1],0)

while (X,Y) not in opened:

    # use key presses to determine how the player moves
    dX,dY = 0,1 # default is to fall
    keys = pew.keys()
    if keys!=0:
        if keys&pew.K_O: # fly up with O
            dY = -1
        # just left and right
        if keys&pew.K_LEFT:
            dX = -1
            dY = 0
        if keys&pew.K_RIGHT:
            dX = +1
            dY = 0
    
    # blank player pixel at old pos
    screen.pixel(X,Y,0)
    # change pos
    if ( Y+dY in range(1,7) and X+dX in range(1,7) ) or ( (X+dX,Y+dY) in opened ):
        X+=dX
        Y+=dY
    # put player pixel at new pos
    screen.pixel(X,Y,3)
    
    pew.show(screen)

    pew.tick(1/6)

Obviously what *should* happen when you pass through a door is that you end up in another room. This is implemented in the program below. The new room will again has open and closed doors, generated randomly by the circuit.

In [ ]:
import pew
from microqiskit import *

pew.init()
screen = pew.Pix()

# set positions of doors
l = (0,4)
r = (7,4)
u = (3,0)
d = (3,7)
doors = [l,r,u,d]

# create a border of B=2 pixels
for A in range(8):
    for B in [0,7]:
        screen.pixel(A,B,2)
        screen.pixel(B,A,2) 

# the player is a B=3 pixel
X,Y = 4,6
screen.pixel(X,Y,3)

# set up the circuit that decides whether doors are open
qc = QuantumCircuit(2,2)

# and those for measurement
meas_zz = QuantumCircuit(2,2)
meas_zz.measure(0,0)
meas_zz.measure(1,1)
meas_xx = QuantumCircuit(2,2)
meas_xx.h(0)
meas_xx.h(1)
meas_xx.measure(0,0)
meas_xx.measure(1,1)

while True:

    # use the results to set which doors are open
    m_zz = simulate(qc+meas_zz,shots=1,get='memory')[0]
    m_xx = simulate(qc+meas_xx,shots=1,get='memory')[0]

    opened = []
    if m_zz[0]=='0':
        opened.append(l)
    if m_zz[1]=='0':
        opened.append(r)
    if m_xx[0]=='0':
        opened.append(u)
    if m_xx[1]=='0':
        opened.append(d)
        
    # set open door pixels to B=0
    for door in doors:
        screen.pixel(door[0],door[1],2)
    for door in opened:
        screen.pixel(door[0],door[1],0)

    while (X,Y) not in opened:

        # use key presses to determine how the player moves
        dX,dY = 0,1 # default is to fall
        keys = pew.keys()
        if keys!=0:
            if keys&pew.K_O: # fly up with O
                dY = -1
            # just left and right
            if keys&pew.K_LEFT:
                dX = -1
                dY = 0
            if keys&pew.K_RIGHT:
                dX = +1
                dY = 0

        # blank player pixel at old pos
        screen.pixel(X,Y,0)
        # change pos
        if ( Y+dY in range(1,7) and X+dX in range(1,7) ) or ( (X+dX,Y+dY) in opened ):
            X+=dX
            Y+=dY
        # put player pixel at new pos
        screen.pixel(X,Y,3)

        pew.show(screen)

        pew.tick(1/6)
        
    if (X,Y)==u:
        X,Y = 3,6
    if (X,Y)==d:
        X,Y = 3,1
    if (X,Y)==l:
        X,Y = 6,4
    if (X,Y)==r:
        X,Y = 1,4

So far, we have always used a blank circuit. This always outputs `00` for z measurements, and so always has the left and right doors open. The other two doors are randomly open or closed.

The aim of the game will be to climb sufficiently high up the tower. For this, you'll want to make sure that the ceiling door is open as much as possible. This can be done by adding a Hadamard to the circuit, but how can we allow the player to do this?

In the following, two randomly moving objects are added. If the bat hits them, a Hadamard is applied to the circuit. For the object near the bottom of the room, it is the Hadamard that controls the bottom door that is applied. The object near the top similarly controls the top door.

In [ ]:
import pew
from microqiskit import *

pew.init()
screen = pew.Pix()

# set positions of doors
l = (0,4)
r = (7,4)
u = (3,0)
d = (3,7)
doors = [l,r,u,d]

# create a border of B=2 pixels
for A in range(8):
    for B in [0,7]:
        screen.pixel(A,B,2)
        screen.pixel(B,A,2) 

# the player is a B=3 pixel
X,Y = 4,6
screen.pixel(X,Y,3)

# set positions of the hadamard objects
H = [[],[]]
H[0] = [6,6]
H[1] = [1,1]

# set up the circuit that decides whether doors are open
qc = QuantumCircuit(2,2)

# and those for measurement
meas_zz = QuantumCircuit(2,2)
meas_zz.measure(0,0)
meas_zz.measure(1,1)
meas_xx = QuantumCircuit(2,2)
meas_xx.h(0)
meas_xx.h(1)
meas_xx.measure(0,0)
meas_xx.measure(1,1)

qrng = QuantumCircuit(2,2)

while True:

    # use the results to set which doors are open
    m_zz = simulate(qc+meas_zz,shots=1,get='memory')[0]
    m_xx = simulate(qc+meas_xx,shots=1,get='memory')[0]

    opened = []
    if m_zz[0]=='0':
        opened.append(l)
    if m_zz[1]=='0':
        opened.append(r)
    if m_xx[0]=='0':
        opened.append(u)
    if m_xx[1]=='0':
        opened.append(d)
        
    # set open door pixels to B=0
    for door in doors:
        screen.pixel(door[0],door[1],2)
    for door in opened:
        screen.pixel(door[0],door[1],0)
        
    frame = 0
    while (X,Y) not in opened:
        
        # randomly move the positions of H[0] and H[1]
        for j in range(2):
            screen.pixel(H[j][0],H[j][1],0)
        m = simulate(qrng+meas_xx,shots=1,get='memory')[0]
        for j in range(2):
            dH = (m[j]=='0') - (m[j]=='1')
            if H[j][0]+dH in range(1,7):
                H[j][0]+=dH
        frame += 1 # brightness flashes, and so depends on frame
        for j in range(2):
            screen.pixel(H[j][0],H[j][1],1+(frame%2)) 

        # use key presses to determine how the player moves
        dX,dY = 0,1 # default is to fall
        keys = pew.keys()
        if keys!=0:
            if keys&pew.K_O: # fly up with O
                dY = -1
            # just left and right
            if keys&pew.K_LEFT:
                dX = -1
                dY = 0
            if keys&pew.K_RIGHT:
                dX = +1
                dY = 0

        # blank player pixel at old pos
        screen.pixel(X,Y,0)
        # change pos
        if ( Y+dY in range(1,7) and X+dX in range(1,7) ) or ( (X+dX,Y+dY) in opened ):
            X+=dX
            Y+=dY
        # put player pixel at new pos
        screen.pixel(X,Y,3)
        
        # if the player is at the same pos as H[0] or H[1]
        # apply the corresponding Hadamard
        for j in range(2):
            if (X,Y)==(H[j][0],H[j][1]):
                qc.h(j)

        pew.show(screen)

        pew.tick(1/6)
        
    if (X,Y)==u:
        X,Y = 3,6
    if (X,Y)==d:
        X,Y = 3,1
    if (X,Y)==l:
        X,Y = 6,4
    if (X,Y)==r:
        X,Y = 1,4

Now we just need to add in an end condition, so that we can actually complete the game. We'll simply make it so that you win when you are ten floors higher than when you started. This then triggers the end screen, which will be some random pixels generated by the circuit.

In [ ]:
import pew
from microqiskit import *

pew.init()
screen = pew.Pix()

# set positions of doors
l = (0,4)
r = (7,4)
u = (3,0)
d = (3,7)
doors = [l,r,u,d]

# create a border of B=2 pixels
for A in range(8):
    for B in [0,7]:
        screen.pixel(A,B,2)
        screen.pixel(B,A,2) 

# the player is a B=3 pixel
X,Y = 4,6
screen.pixel(X,Y,3)

# set how high the player has gone up the tower
height = 0

# set positions of the hadamard objects
H = [[],[]]
H[0] = [6,6]
H[1] = [1,1]

# set up the circuit that decides whether doors are open
qc = QuantumCircuit(2,2)

# and those for measurement
meas_zz = QuantumCircuit(2,2)
meas_zz.measure(0,0)
meas_zz.measure(1,1)
meas_xx = QuantumCircuit(2,2)
meas_xx.h(0)
meas_xx.h(1)
meas_xx.measure(0,0)
meas_xx.measure(1,1)

qrng = QuantumCircuit(2,2)

while height<10:

    # use the results to set which doors are open
    m_zz = simulate(qc+meas_zz,shots=1,get='memory')[0]
    m_xx = simulate(qc+meas_xx,shots=1,get='memory')[0]

    opened = []
    if m_zz[0]=='0':
        opened.append(l)
    if m_zz[1]=='0':
        opened.append(r)
    if m_xx[0]=='0':
        opened.append(u)
    if m_xx[1]=='0':
        opened.append(d)
        
    # set open door pixels to B=0
    for door in doors:
        screen.pixel(door[0],door[1],2)
    for door in opened:
        screen.pixel(door[0],door[1],0)
        
    frame = 0
    while (X,Y) not in opened:
        
        # randomly move the positions of H[0] and H[1]
        for j in range(2):
            screen.pixel(H[j][0],H[j][1],0)
        m = simulate(qrng+meas_xx,shots=1,get='memory')[0]
        for j in range(2):
            dH = (m[j]=='0') - (m[j]=='1')
            if H[j][0]+dH in range(1,7):
                H[j][0]+=dH
        frame += 1 # brightness flashes, and so depends on frame
        for j in range(2):
            screen.pixel(H[j][0],H[j][1],1+(frame%2)) 

        # use key presses to determine how the player moves
        dX,dY = 0,1 # default is to fall
        keys = pew.keys()
        if keys!=0:
            if keys&pew.K_O: # fly up with O
                dY = -1
            # just left and right
            if keys&pew.K_LEFT:
                dX = -1
                dY = 0
            if keys&pew.K_RIGHT:
                dX = +1
                dY = 0

        # blank player pixel at old pos
        screen.pixel(X,Y,0)
        # change pos
        if ( Y+dY in range(1,7) and X+dX in range(1,7) ) or ( (X+dX,Y+dY) in opened ):
            X+=dX
            Y+=dY
        # put player pixel at new pos
        screen.pixel(X,Y,3)
        
        # if the player is at the same pos as H[0] or H[1]
        # apply the corresponding Hadamard
        for j in range(2):
            if (X,Y)==(H[j][0],H[j][1]):
                qc.h(j)

        pew.show(screen)

        pew.tick(1/6)
        
    if (X,Y)==u:
        X,Y = 3,6
        height += 1
    if (X,Y)==d:
        X,Y = 3,1
        height -= 1
    if (X,Y)==l:
        X,Y = 6,4
    if (X,Y)==r:
        X,Y = 1,4
        
while True:
    for x in range(4):
        for y in range(4):
            m_zz = simulate(qc+meas_zz,shots=1,get='memory')[0]
            m_xx = simulate(qc+meas_xx,shots=1,get='memory')[0]
            screen.pixel(x,y,1+2*(m_zz[0]=='0'))
            screen.pixel(x+4,y,1+2*(m_zz[1]=='0'))
            screen.pixel(x,y+4,1+2*(m_xx[0]=='0'))
            screen.pixel(x+4,y+4,1+2*(m_xx[1]=='0'))
    pew.show(screen)
    pew.tick(1/6)

And there we have it! A simple game, made using quantum programming.

**[Click here to return to the index](Start-Here.ipynb)**